In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import torchmetrics

import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.autograd import Variable

import transformers
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel

In [2]:
# choose train mode
global mode
# mode = 'pretrain'
mode = 'finetune'
# mode = 'else'


In [3]:
torch.cuda.device_count()



2

In [4]:
# device and tokenizer

# device = torch.device('cuda:0,1')


tokenizer = GPT2Tokenizer('./vocab_file/vocab.json', './vocab_file/merges.txt')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
if mode == 'pretrain':
    tokenizer.save_pretrained('../save_model/benchmarks_models/train_raw_APD3_500epochs_model/') # when pretrain model
print(tokenizer)

PreTrainedTokenizer(name_or_path='', vocab_size=23, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '[PAD]'})


In [5]:
#定义数据集
class Dataset(torch.utils.data.Dataset):

    def __init__(self):
        if mode == 'finetune':
#             with open('../data/benchmarks_data/bigclustr09.txt') as f:  # when finetune
            with open('../data/benchmarks_data/nature_AMPs10-64.txt') as f:  # when finetune
                lines = f.readlines()
        elif mode == 'pretrain':
            with open('../data/ADP3_amp.txt') as f:
#             with open('./data/pretrain_data/uniprot10-63.txt') as f:  # when pretrain model 
                lines = f.readlines()
        else:
            print('train mode error')
            with open('./data/ADP3_amp.txt') as f:
                lines = f.readlines()
        lines = [i.strip() for i in lines]

        self.lines = lines

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, i):
        return self.lines[i]


global val_split
if mode == 'pretrian':
    val_split = 0.1
elif mode =='finetune':
    val_split = 0.1
else:
    val_split = 0.1

shuffle_dataset = True
random_seed = 42

dataset = Dataset()
dataset_size = len(dataset)

indices = list(range(dataset_size))
split = int(np.floor(val_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = Data.SubsetRandomSampler(train_indices)
val_sampler = Data.SubsetRandomSampler(val_indices)

def collate_fn(data):
    data = tokenizer.batch_encode_plus(data,
                                       padding=True,
                                       truncation=True,
                                       max_length=64,
                                       return_tensors='pt')

    data['labels'] = data['input_ids'].clone()

    return data



train_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=64, 
    sampler=train_sampler,
    collate_fn=collate_fn,
    drop_last=True,)


val_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=64, 
    sampler=val_sampler,
    collate_fn=collate_fn,
    drop_last=True,)

# for i, data in enumerate(val_loader):
    

#     for k, v in data.items():
#         print(k, v.shape, v)

len(train_loader)

105

In [6]:
# define GPT model

from transformers import GPT2Model, GPT2Config

# Initializing a GPT2 configuration
configuration = GPT2Config(n_layer=12, 
                           n_head=12,
                           n_embd=768)

print(configuration)

# Initializing a model from the configuration
if mode == 'pretrain':
    model = GPT2LMHeadModel(configuration)  # when pretrain model
elif mode == 'finetune':
    model = GPT2LMHeadModel.from_pretrained('../save_model/pretrain_model/pretrained-GPT-10-64washed-30epochs/')
else:
    pass
#     model = GPT2LMHeadModel.from_pretrained('./save_model/pretrain_model/pretrained-GPT-10-64raw-20epochs/')
# model = torch.load('./save_model/pretrained-gpt-10-48-30epochs/pytorch_model.bin')  # pretrain model use
# model = nn.DataParallel(model)

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.21.1",
  "use_cache": true,
  "vocab_size": 50257
}



In [7]:
from torch.optim import AdamW
from transformers.optimization import get_scheduler
from torchmetrics import Accuracy 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    device_ids = [0, 1]  # 使用第一和第二个GPU
else:
    device_ids = None
    
accuracy = Accuracy(task="multiclass",num_classes=2300,ignore_index=23)
accuracy = accuracy.to(device) 

epochs = 125

train_loss_list = []
val_loss_list = []
train_acc_list =[]
val_acc_list = []
    
#训练
def train():
    global model

    model = model.to(device)
#     model = nn.DataParallel(model, device_ids=device_ids)
    optimizer = AdamW(model.parameters(), lr=1e-7)
    scheduler = get_scheduler(name='linear',
                              num_warmup_steps=400,
                              num_training_steps=len(train_loader)*epochs,
                              optimizer=optimizer)

    model.train()
    print('开始训练')
    start_time = time.time()

    for epoch in range(epochs):
        train_loss = []
        train_accuracy = []
        val_loss = []
        val_accuracy = []
        for batch_idx, batch_data in enumerate(train_loader):
            batch_data = batch_data.to(device)
            out = model(**batch_data)
            loss = out['loss']
    
            train_loss.append(loss.item())
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()

            optimizer.zero_grad()
            model.zero_grad()
            
            labels = batch_data['labels'][:, 1:]
            out = out['logits'].argmax(dim=2)[:, :-1]
            train_acc = accuracy(out, labels)
            train_accuracy.append(train_acc.tolist())
            lr = optimizer.state_dict()['param_groups'][0]['lr']
            
            if batch_idx % 50 == 0:
                print('train_batch: {:3d}  loss:{:.4f}  accuracy:{:.4f}'
                      .format(batch_idx, loss.item(), train_acc.item()))
            
        for batch_idx, batch_data in enumerate(val_loader):
            batch_data = batch_data.to(device)
            out = model(**batch_data)
            loss = out['loss']
            labels = batch_data['labels'][:, 1:]
            out = out['logits'].argmax(dim=2)[:, :-1]
            val_acc = accuracy(out, labels)
            val_loss.append(loss.item())
            val_accuracy.append(val_acc.tolist())
            
            if batch_idx % 50 == 0:
                print('val_batch:   {:3d}  loss:{:.4f}  accuracy:{:.4f}'
                      .format(batch_idx, loss.item(), val_acc.item()))
            
        train_loss_list.append(np.mean(train_loss))
        train_acc_list.append(np.mean(train_accuracy))
        val_loss_list.append(np.mean(val_loss))
        val_acc_list.append(np.mean(val_accuracy))
        
        train_time = time.time()
        print('第{}代训练完成,历时{}秒'.format(epoch+1,train_time-start_time))
        print('epoch {} mean training loss:{:.4f}'.format(epoch+1, np.mean(train_loss)))
        print('epoch {} mean training accuracy:{:.4f}'.format(epoch+1, np.mean(train_accuracy)))
        print('epoch {} mean val loss:{:.4f}'.format(epoch+1, np.mean(val_loss)))
        print('epoch {} mean val accuracy:{:.4f} '.format(epoch+1, np.mean(val_accuracy)))
        print(' ')
        
    
    end_time = time.time()
    print('训练结束,训练时长：',end_time-start_time, '秒')   
    
    

In [ ]:
# 训练模型

train()


开始训练
train_batch:   0  loss:1.5129  accuracy:0.1348
train_batch:  50  loss:1.5532  accuracy:0.1328


In [ ]:
# 保存模型

model = model.to('cpu')

if mode == 'pretrain':
    model.save_pretrained('../save_model/benchmarks_models/train_raw_APD3_125epochs_model/')  # when pretrain model
elif mode == 'finetune':
    torch.save(model, '../save_model/benchmarks_models/nature_AMPs-125epochs_model')  # finetune model
else:
    pass
    

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_acc(save_dir, mode, epochs):
    plt.figure(figsize=(12, 12),dpi=300,)

    # Training and Validation Loss
    plt.subplot(2, 1, 1)
    plt.plot(range(1, len(train_loss_list) + 1), train_loss_list, label='Training Loss', linestyle='-', marker='o')
    plt.plot(range(1, len(val_loss_list) + 1), val_loss_list, label='Validation Loss', linestyle='--', marker='x')
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    plt.title('Training and Validation Loss', fontsize=16)
    plt.grid(True)
    plt.legend(fontsize=12)

    # Training and Validation Accuracy
    plt.subplot(2, 1, 2)
    plt.plot(range(1, len(train_acc_list) + 1), train_acc_list, label='Training Accuracy', linestyle='-', marker='o')
    plt.plot(range(1, len(val_acc_list) + 1), val_acc_list, label='Validation Accuracy', linestyle='--', marker='x')
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.title('Training and Validation Accuracy', fontsize=16)
    plt.grid(True)
    plt.legend(fontsize=12)

    plt.tight_layout()
    plt.savefig(f"{save_dir}{mode}{epochs}_Training_and_Validation_Loss_and_Accuracy.pdf")
    plt.show()

# Example usage (note: 'mode' and 'epochs' should be defined)
save_dir = '/home/xms/AMP-master/save_model/benchmarks_models/training_pdf/'
plot_loss_acc(save_dir, 'nature_AMPs-', '125')


In [ ]:
# import matplotlib.pyplot as plt
# def plot_loss_acc(save_dir, mode, epochs):
#     plt.figure(figsize=(12, 12))

#     plt.subplot(2, 2, 1)
#     plt.plot(range(1, len(train_loss_list) + 1), train_loss_list, label='train_loss')
#     plt.plot(range(1, len(val_loss_list) + 1), val_loss_list, label='val_loss')
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.title('Training and Validation Loss')
#     plt.legend()

#     plt.subplot(2, 2, 2)
#     plt.plot(range(1, len(train_acc_list) + 1), train_acc_list, label='train_acc')
#     plt.plot(range(1, len(val_acc_list) + 1), val_acc_list, label='val_acc')
#     plt.xlabel('Epochs')
#     plt.ylabel('Accuracy')
#     plt.title('Training and Validation Accuracy')
#     plt.legend()

#     plt.tight_layout()
#     plt.savefig(save_dir+mode+str(epochs)+'_Training and Validation Loss and Accuracy')  # 保存损失和准确率曲线图
#     plt.show()
    
# save_dir = '/home/xms/AMP-master/generate_file/train_acc_loss/'
# plot_loss_acc(save_dir, mode, epochs)

In [ ]:
# # 结果作图

# import matplotlib.pyplot as plt

# x1 = [(x+1) for x in range(len(train_loss_list))]
# x2 = [(x+1) for x in range(len(val_acc_list))]
# y1 = train_loss_list
# y2 = train_acc_list
# y3 = val_loss_list
# y4 = val_acc_list

# plt.plot(x1, y1, label="AMP training loss")
# plt.plot(x1, y3, label="AMP val_loss")
# plt.xlabel('step')
# plt.ylabel('loss')
# plt.title('AMP train losses show')
# plt.legend()
# if mode == 'pretrain':
#     plt.savefig('/xms/AMP-master/generate_file/train_graphical_result-2023/pretrain_result/'+
#                 mode+'10-48washed-'+str(epochs)+'epochs_loss.jpg')
# elif mode == 'finetune':
#     plt.savefig('/home/xms/AMP-master/generate_file/train_acc_loss/'+
#                 mode+'10-64washed-'+str(epochs)+'epochs_loss.jpg')
# else:
#     pass
# plt.show()


# plt.plot(x2, y2, label="AMP train_acc curse")
# plt.plot(x2, y4, label="AMP val_acc curse")
# plt.xlabel('step')
# plt.ylabel('acc')
# plt.title('AMP val_acc show')
# plt.legend()
# if mode == 'pretrain':
#     plt.savefig('/xms/AMP-master/generate_file/train_graphical_result-2023/pretrain_result/'+
#                 mode+'10-48washed-'+str(epochs)+'epochs_accuracy.jpg')
# elif mode == 'finetune':
#     plt.savefig('/home/xms/AMP-master/generate_file/train_acc_loss/'+
#                 mode+'10-64washed-'+str(epochs)+'epochs_accuracy.jpg')
# else:
#     pass

# plt.show()